# BBC News Category Classification

### Exploring the Data

In [2]:
import pandas as pd
data=pd.read_csv("https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv")
data

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
...,...,...
2220,business,cars pull down us retail figures us retail sal...
2221,politics,kilroy unveils immigration policy ex-chatshow ...
2222,entertainment,rem announce new glasgow concert us band rem h...
2223,politics,how political squabbles snowball it s become c...


In [3]:
data.category.value_counts()

sport            511
business         510
politics         417
tech             401
entertainment    386
Name: category, dtype: int64

The data consists of BBC news stories and their classification into categories. There are 2225 observations. There are five categories in the target variable of this dataset. Out of 2225 categories the most are sport (511) and politics (510) and the fewest of the news fall into the category of entertainment (386)

### Preprocessing the Data

There are two steps I need to take while preprocessing this data.

1) The independet variable, the actual news stories, are all cut after the first 100 words and the top 10,000 words are tokenized.

2) The dependent variable needs to be either one hot encoded or be obtained the dummies for. I prefer to use pandas to get dummies rather than using Keras.

In [4]:
text = data['text']

category=data['category']

In [5]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 100
max_words = 10000  

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(text)
sequences = tokenizer.texts_to_sequences(text) 

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

text = pad_sequences(sequences, maxlen=maxlen)

category = np.asarray(category)

print('Shape of data tensor:',text.shape)
print('Shape of category tensor:',category.shape)



Using TensorFlow backend.


Found 29726 unique tokens.
Shape of data tensor: (2225, 100)
Shape of category tensor: (2225,)


In [6]:
X=text
X
y=category
y

y=pd.get_dummies(y)

display(y)

,business,entertainment,politics,sport,tech
0,0,0,0,0,1
1,1,0,0,0,0
2,0,0,0,1,0
3,0,0,0,1,0
4,0,1,0,0,0
...,...,...,...,...,...
2220,1,0,0,0,0
2221,0,0,1,0,0
2222,0,1,0,0,0
2223,0,0,1,0,0


### Model A: A Model with an embedding layer and two dense layers (with no layers meant for sequential data)

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

print(X_train.shape)
print(y_train.shape)

(1668, 100)
(1668, 5)


In [8]:
from keras.layers import Dense, Embedding
from keras.models import Sequential
from keras.layers import Flatten, Dense

model = Sequential()
model.add(Embedding(10000, 8, input_length=maxlen))

model.add(Flatten())
model.add(Dense(5, activation='sigmoid'))
model.add(Dense(5, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=32)

score, acc = model.evaluate(X_test, y_test)
print('Test score:', score)
print('Test accuracy:', acc)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 8)            80000     
_________________________________________________________________
flatten_2 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 4005      
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 30        
Total params: 84,035
Trainable params: 84,035
Non-trainable params: 0
_________________________________________________________________


C:\Users\qures\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
1668/1668 [==============================] - 0s 250us/step - loss: 0.6696 - acc: 0.5253
Epoch 2/10
1668/1668 [==============================] - 0s 125us/step - loss: 0.5806 - acc: 0.6897
Epoch 3/10
1668/1668 [==============================] - 0s 134us/step - loss: 0.5504 - acc: 0.7379
Epoch 4/10
1668/1668 [==============================] - 0s 123us/step - loss: 0.5293 - acc: 0.8000
Epoch 5/10
1668/1668 [==============================] - 0s 112us/step - loss: 0.5055 - acc: 0.8008
Epoch 6/10
1668/1668 [==============================] - 0s 106us/step - loss: 0.4795 - acc: 0.8089
Epoch 7/10
1668/1668 [==============================] - 0s 110us/step - loss: 0.4542 - acc: 0.8183
Epoch 8/10
1668/1668 [==============================] - 0s 93us/step - loss: 0.4326 - acc: 0.8237
Epoch 9/10
1668/1668 [==============================] - 0s 100us/step - loss: 0.4144 - acc: 0.8266
Epoch 10/10
557/557 [==============================] - 0s 132us/step
Test score: 0.41835008617791497
Test accu

In the above model, I used embedding and two dense layers to run the model A. The accuracy of the model is 81.58%. 

### Model B: A model using an Embedding layer with Conv1d Layers

In [11]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

max_features = 10000  
max_len = 100 

model = Sequential()
model.add(layers.Embedding(max_features, 128, input_length=max_len))
model.add(layers.Conv1D(32, 7, activation='relu')) 
model.add(layers.MaxPooling1D(5)) #
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(5))

model.summary()

model.compile(optimizer=RMSprop(lr=1e-4),
              loss='binary_crossentropy',
              metrics=['acc'])

history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2)

score, acc = model.evaluate(X_test, y_test)
print('Test score:', score)
print('Test accuracy:', acc)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 128)          1280000   
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 94, 32)            28704     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 18, 32)            0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 12, 32)            7200      
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 165       
Total params: 1,316,069
Trainable params: 1,316,069
Non-trainable params: 0
____________________________________________

C:\Users\qures\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1334 samples, validate on 334 samples
Epoch 1/10
1334/1334 [==============================] - 1s 1ms/step - loss: 1.8814 - acc: 0.8000 - val_loss: 1.7397 - val_acc: 0.8000
Epoch 2/10
1334/1334 [==============================] - 1s 848us/step - loss: 1.6143 - acc: 0.8000 - val_loss: 1.7158 - val_acc: 0.8000
Epoch 3/10
1334/1334 [==============================] - 1s 815us/step - loss: 1.5890 - acc: 0.8000 - val_loss: 1.7015 - val_acc: 0.8000
Epoch 4/10
1334/1334 [==============================] - 1s 817us/step - loss: 1.5723 - acc: 0.8000 - val_loss: 1.6912 - val_acc: 0.8000
Epoch 5/10
1334/1334 [==============================] - 1s 799us/step - loss: 1.5596 - acc: 0.8000 - val_loss: 1.6881 - val_acc: 0.8000
Epoch 6/10
1334/1334 [==============================] - 1s 799us/step - loss: 1.5496 - acc: 0.8000 - val_loss: 1.6823 - val_acc: 0.8000
Epoch 7/10
1334/1334 [==============================] - 1s 831us/step - loss: 1.5417 - acc: 0.8000 - val_loss: 1.6790 - val_acc: 0.8000
Epo

The above model with embedding and two Conv1D layers yeilds 80% accuracy in the model. 

### Model C: A model using an Embedding layer with one LSTM sequential layer

In [24]:
from keras.layers import LSTM, Embedding, Dense
model_c = Sequential()
model_c.add(Embedding(10000, 32))
model_c.add(LSTM(32))
model_c.add(Dense(5, activation='sigmoid'))

model_c.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model_c.fit(X_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

score, acc = model_c.evaluate(X_test, y_test)
print('Test score:', score)
print('Test accuracy:', acc)

C:\Users\qures\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1334 samples, validate on 334 samples
Epoch 1/10
1334/1334 [==============================] - 6s 4ms/step - loss: 0.5368 - acc: 0.7933 - val_loss: 0.4998 - val_acc: 0.8000
Epoch 2/10
1334/1334 [==============================] - 5s 4ms/step - loss: 0.4971 - acc: 0.8000 - val_loss: 0.4930 - val_acc: 0.8000
Epoch 3/10
1334/1334 [==============================] - 5s 4ms/step - loss: 0.4818 - acc: 0.8000 - val_loss: 0.4700 - val_acc: 0.8000
Epoch 4/10
1334/1334 [==============================] - 5s 4ms/step - loss: 0.4366 - acc: 0.8127 - val_loss: 0.4298 - val_acc: 0.8287
Epoch 5/10
1334/1334 [==============================] - 5s 4ms/step - loss: 0.3810 - acc: 0.8339 - val_loss: 0.3821 - val_acc: 0.8341
Epoch 6/10
1334/1334 [==============================] - 5s 3ms/step - loss: 0.3321 - acc: 0.8387 - val_loss: 0.3333 - val_acc: 0.8461
Epoch 7/10
1334/1334 [==============================] - 5s 4ms/step - loss: 0.2935 - acc: 0.8856 - val_loss: 0.3198 - val_acc: 0.8707
Epoch 8/10
1334

The above model with a single LSTM model yeilds 90% accuracy. Which is substantially higher than the previous two models. 

### Model D: A model using an Embedding layer with stacked LSTM sequential layers

In [22]:
model_d = Sequential()
model_d.add(Embedding(10000, 32))
model_d.add(LSTM(32, return_sequences=True))
model_d.add(LSTM(32, return_sequences=True))
model_d.add(LSTM(32))
model_d.add(Dense(5, activation='sigmoid'))

model_d.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model_d.fit(X_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

score, acc = model_d.evaluate(X_test, y_test)
print('Test score:', score)
print('Test accuracy:', acc)

C:\Users\qures\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1334 samples, validate on 334 samples
Epoch 1/10
1334/1334 [==============================] - 13s 10ms/step - loss: 0.5209 - acc: 0.7925 - val_loss: 0.4979 - val_acc: 0.8000
Epoch 2/10
1334/1334 [==============================] - 11s 8ms/step - loss: 0.4530 - acc: 0.8136 - val_loss: 0.3903 - val_acc: 0.8347
Epoch 3/10
1334/1334 [==============================] - 11s 8ms/step - loss: 0.3828 - acc: 0.8319 - val_loss: 0.3529 - val_acc: 0.8389
Epoch 4/10
1334/1334 [==============================] - 10s 8ms/step - loss: 0.3359 - acc: 0.8499 - val_loss: 0.3505 - val_acc: 0.8449
Epoch 5/10
1334/1334 [==============================] - 11s 8ms/step - loss: 0.3044 - acc: 0.8663 - val_loss: 0.3876 - val_acc: 0.8383
Epoch 6/10
1334/1334 [==============================] - 11s 8ms/step - loss: 0.2858 - acc: 0.8711 - val_loss: 0.3408 - val_acc: 0.8431
Epoch 7/10
1334/1334 [==============================] - 10s 7ms/step - loss: 0.2644 - acc: 0.8780 - val_loss: 0.4208 - val_acc: 0.8168
Epoch 8

In the above model, I have three LSTM layers, we see that the accuracy of the model (83) decreases considerably from model C.

### Model E: A model using an Embedding layer with bidirectional sequential layers

In [23]:
model_e = Sequential()
model_e.add(layers.Embedding(max_features, 32))
model_e.add(layers.Bidirectional(layers.LSTM(32)))
model_e.add(layers.Dense(5, activation='sigmoid'))

model_e.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model_e.fit(X_train, y_train, epochs=10, batch_size=128, validation_split=0.2)

score, acc = model_e.evaluate(X_test, y_test)
print('Test score:', score)
print('Test accuracy:', acc)

C:\Users\qures\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1334 samples, validate on 334 samples
Epoch 1/10
1334/1334 [==============================] - 4s 3ms/step - loss: 0.6396 - acc: 0.7643 - val_loss: 0.5036 - val_acc: 0.8000
Epoch 2/10
1334/1334 [==============================] - 3s 2ms/step - loss: 0.5000 - acc: 0.8000 - val_loss: 0.4976 - val_acc: 0.8000
Epoch 3/10
1334/1334 [==============================] - 2s 2ms/step - loss: 0.4966 - acc: 0.8000 - val_loss: 0.4965 - val_acc: 0.8000
Epoch 4/10
1334/1334 [==============================] - 2s 2ms/step - loss: 0.4946 - acc: 0.8000 - val_loss: 0.4954 - val_acc: 0.8000
Epoch 5/10
1334/1334 [==============================] - 2s 2ms/step - loss: 0.4908 - acc: 0.8000 - val_loss: 0.4913 - val_acc: 0.8000
Epoch 6/10
1334/1334 [==============================] - 2s 1ms/step - loss: 0.4815 - acc: 0.8000 - val_loss: 0.4832 - val_acc: 0.8000
Epoch 7/10
1334/1334 [==============================] - 2s 1ms/step - loss: 0.4641 - acc: 0.8000 - val_loss: 0.4633 - val_acc: 0.8000
Epoch 8/10
1334

In the above model we have one bidirectional sequential layer and results in the accuracy of 85%

### Optimizing the best model (Model C) with drop outs

In [27]:
model = Sequential()
model.add(Embedding(10000, 32))
model.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2)) 
model.add(Dense(5, activation='sigmoid'))


model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

score, acc = model.evaluate(X_test, y_test,
                            batch_size=32)
print('Test score:', score)
print('Test accuracy:', acc)

C:\Users\qures\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1334 samples, validate on 334 samples
Epoch 1/10
1334/1334 [==============================] - 3s 3ms/step - loss: 0.5446 - acc: 0.7774 - val_loss: 0.4985 - val_acc: 0.8000
Epoch 2/10
1334/1334 [==============================] - 3s 2ms/step - loss: 0.4996 - acc: 0.8000 - val_loss: 0.4976 - val_acc: 0.8000
Epoch 3/10
1334/1334 [==============================] - 3s 2ms/step - loss: 0.4977 - acc: 0.8000 - val_loss: 0.4967 - val_acc: 0.8000
Epoch 4/10
1334/1334 [==============================] - 3s 2ms/step - loss: 0.4931 - acc: 0.8000 - val_loss: 0.4912 - val_acc: 0.8000
Epoch 5/10
1334/1334 [==============================] - 3s 2ms/step - loss: 0.4818 - acc: 0.8000 - val_loss: 0.4805 - val_acc: 0.8000
Epoch 6/10
1334/1334 [==============================] - 3s 2ms/step - loss: 0.4610 - acc: 0.8000 - val_loss: 0.4561 - val_acc: 0.8000
Epoch 7/10
1334/1334 [==============================] - 3s 2ms/step - loss: 0.4276 - acc: 0.8021 - val_loss: 0.4204 - val_acc: 0.8012
Epoch 8/10
1334

Retraining model C using a dropout leads to a decreased accuracy of 87%. Below I rerun the model using increased epochs. 

### Optimizing the best model (Model C) with increased epochs

In [28]:
model = Sequential()
model.add(Embedding(10000, 32))
model.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2)) 
model.add(Dense(5, activation='sigmoid'))


model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train,
                    epochs=20,
                    batch_size=32,
                    validation_split=0.2)

score, acc = model.evaluate(X_test, y_test)
print('Test score:', score)
print('Test accuracy:', acc)

C:\Users\qures\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1334 samples, validate on 334 samples
Epoch 1/20
1334/1334 [==============================] - 3s 2ms/step - loss: 0.5435 - acc: 0.7825 - val_loss: 0.4972 - val_acc: 0.8000
Epoch 2/20
1334/1334 [==============================] - 3s 2ms/step - loss: 0.5002 - acc: 0.8000 - val_loss: 0.4968 - val_acc: 0.8000
Epoch 3/20
1334/1334 [==============================] - 3s 2ms/step - loss: 0.4978 - acc: 0.8000 - val_loss: 0.4959 - val_acc: 0.8000
Epoch 4/20
1334/1334 [==============================] - 3s 2ms/step - loss: 0.4944 - acc: 0.8000 - val_loss: 0.4922 - val_acc: 0.8000
Epoch 5/20
1334/1334 [==============================] - 3s 2ms/step - loss: 0.4860 - acc: 0.8000 - val_loss: 0.4832 - val_acc: 0.8000
Epoch 6/20
1334/1334 [==============================] - 3s 2ms/step - loss: 0.4656 - acc: 0.8000 - val_loss: 0.4614 - val_acc: 0.8006
Epoch 7/20
1334/1334 [==============================] - 3s 2ms/step - loss: 0.4276 - acc: 0.8106 - val_loss: 0.4102 - val_acc: 0.8234
Epoch 8/20
1334

Increasing epochs helped to increase the accuracy to a 90.48% but it is still not equivalent to the 92.78% in the original model.

### Discuss

Model C, where one embedding and one LSTM sequential layer was used, performed the best. This is not what I had expected as I imagined adding more sequential layers would make the model perform better. 

However all the subsequent models; biderectional LSTM, multiple LSTM layers and dropouts made the model perform worse. This might be due to the fact that the dataset is not very large.I would like to try using Glove embeddings or GRU in order to see if a diffferent methodology would provide a better and more accurate model. I have seen that multiple Conv1D layers and multiple LSTM by themselves do not improve the model. Perhaps combining them would improve our predictive power.